# Simulation processing: step 1

Goal: take the simulated hadrons and create an output file for fj clustering

In [44]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import vector

In [45]:
simulation_dir = '/global/cfs/cdirs/m3246/ewitkowski/cmssim'

muon_prefix = "recoMuons_muons__RECO_obj"
muon_vars = ["pfIsolationR03__sumChargedHadronPt", "pfIsolationR03__sumChargedHadronPt", "pfIsolationR03__sumNeutralHadronEt",
             "pfIsolationR03__sumPhotonEt", "pfIsolationR03__sumPUPt", "pfIsolationR04__sumPhotonEt", "pfIsolationR04__sumPUPt",
             "pfIsolationR04__sumChargedHadronPt", "pfIsolationR04__sumNeutralHadronEt"]

tower_prefix = "recoPFCandidates_particleFlow__RECO_obj"
tower_vars = ["eta_", "pdgId_", "phi_", "pt_"]


In [46]:
muon_iso_dict = {}
tower_dict = {}

for muon_var in muon_vars:
    muon_iso_dict[muon_var] = h5py.File(f'{simulation_dir}/{muon_prefix}_{muon_var}.h5', 'r')['values']

for tower_var in tower_vars:
    tower_dict[tower_var] = h5py.File(f'{simulation_dir}/{tower_prefix}_{tower_var}.h5', 'r')['values']


In [47]:
muon_mass = 0.1056583755 # GeV
particles_to_fastjet = [22, 130, 211, 1, 2]


start_read, stop_read, chunk_size = 0, 100, 100
update_freq = int(chunk_size/10.0)



In [59]:
current_chunk_start = start_read

while current_chunk_start < stop_read:
    
    current_chunk_stop = current_chunk_start + chunk_size
    print(f"Processing chunk from {current_chunk_start} to {current_chunk_stop}...")

    outfile_dimuons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_only_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"
    #outfile_dimuons_isolation = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_iso_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"
    outfile_hadrons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_pre_fj/hadrons_only_{current_chunk_start}_{current_chunk_stop}_cmssim.dat"

    ofile_muons = open(outfile_dimuons, "w")
    #ofile_iso = open(outfile_dimuons_isolation, "w")
    ofile_hadrons = open(outfile_hadrons, "w")
    num_rejects = 0
   
    for event in range(current_chunk_start, current_chunk_stop):
        
        loc_tower, loc_muon_iso = {}, {}

        if event % update_freq == 0:
            print(f"On event {event}...")

        # get the nonmuons
        loc_particle_exists = (tower_dict["pt_"][event] != 0)
        
        for tower_var in tower_vars:
            loc_tower[tower_var] = tower_dict[tower_var][event][loc_particle_exists]
        #print(tower_dict["pdgId_"][event])
        #for muon_var in muon_vars:
            #print(muon_var, np.sum(muon_iso_dict[muon_var][event]!=0))
            #loc_muon_iso[muon_var] = muon_iso_dict[muon_var][event][loc_particle_exists]
            
        # check that there is at least one muon and one antimuon
        num_muons = np.sum(loc_tower["pdgId_"]==13)
        num_amuons = np.sum(loc_tower["pdgId_"]==-13)
        
        if (num_muons < 1) or (num_amuons < 1):
            num_rejects += 1
            
        else:
            # write out the event
            
            # find the hardest muon / antimuon
            loc_mu_pt = loc_tower["pt_"][loc_tower["pdgId_"]==13]
            max_muon_pt = np.max(loc_mu_pt)
            hardest_muon_id = np.where(loc_tower["pt_"]==max_muon_pt)[0][0]
            
            loc_amu_pt = loc_tower["pt_"][loc_tower["pdgId_"]==-13]
            max_amuon_pt = np.max(loc_amu_pt)
            hardest_amuon_id = np.where(loc_tower["pt_"]==max_amuon_pt)[0][0]
            
            
            # sanity check -- just in case there are 2 particles with the same pt...
            if (loc_tower["pdgId_"][hardest_muon_id] != 13) or (loc_tower["pdgId_"][hardest_amuon_id] != -13):
                print("Something very wrong")
                exit()
            
            
            ofile_muons.write("#BEGIN\n")
            #ofile_iso.write("#BEGIN\n")
            ofile_hadrons.write("#BEGIN\n")

            # get the muons
            # construct the muon 4-vector
            mu_1 = vector.obj(pt = loc_tower["pt_"][hardest_muon_id], eta = loc_tower["eta_"][hardest_muon_id], phi = loc_tower["phi_"][hardest_muon_id], M = muon_mass)
            mu_2 = vector.obj(pt = loc_tower["pt_"][hardest_amuon_id], eta = loc_tower["eta_"][hardest_amuon_id], phi = loc_tower["phi_"][hardest_amuon_id], M = muon_mass)
            dimu_system = mu_1 + mu_2

            ofile_muons.write(f"{dimu_system.pt} {dimu_system.eta} {dimu_system.phi} {dimu_system.M}\n")

            # get the hadrons
            for particle_i in range(len(loc_tower["pt_"])):
                if np.abs(loc_tower["pdgId_"][particle_i]) in particles_to_fastjet:
                    particle_vector = vector.obj(pt = loc_tower["pt_"][particle_i], eta = loc_tower["eta_"][particle_i], phi = loc_tower["phi_"][particle_i], M = 0)
                    ofile_hadrons.write(f"{particle_vector.px} {particle_vector.py} {particle_vector.pz} {particle_vector.E}\n")

            ofile_muons.write("#END\n")
            #ofile_iso.write("#END\n")
            ofile_hadrons.write("#END\n")


    ofile_muons.close()  
    #ofile_iso.close() 
    ofile_hadrons.close()
    
    print(f"Done processing chunk.")
    print(f"{num_rejects} events without 2 muons")
    print("\n")
    
    current_chunk_start += chunk_size

print("Done completely!")



                                         

Processing chunk from 0 to 100...
On event 0...
4.5737083579966225 4.5737083326699235
16.74522026330692 16.745221143446543
2.3588291574820346 2.3588291451677748
2.2028315837969927 2.2028315953980524
5.4778401660769305 5.477839235691363
3.5140279960538123 3.5140283728853774
14.607661269033144 14.60766051199522
29.522512735026112 29.522516040562778
38.60944156634855 38.609437286843104
1.2909207045684212 1.2909208020734155
77.91148620263357 77.91151206067661
2.6081635343267067 2.6081634238062747
54.64895110380735 54.64895642151623
6.618156203498984 6.618157326003939
22.734507988258883 22.73451182063218
2.730126325629101 2.7301264119014377
2.264532503911653 2.2645327114118032
2.258489947943886 2.2584898537110263
29.258030763481656 29.258028109688894
71.45073569193482 71.45074339096027
7.053901870615164 7.05390106725099
8.670587279667473 8.670586938656072
27.318050888818334 27.318042184787373
49.34261192173676 49.34261398880358
84.46284824761915 84.4628584367465
70.74219274317511 70.7421819